In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/agents-intensive-capstone-project/Hackathon dataset.txt


In [7]:
# Cell 1: Install required libraries
!pip install -q -U google-genai
!pip install -q requests


In [8]:
# Cell 2: Imports, config, logging, and global state

import os
import json
import logging
from dataclasses import dataclass, field
from typing import List, Dict, Any, Optional
from datetime import datetime

import requests
from google import genai
from google.genai import types as genai_types

# --- Kaggle Secrets: get GOOGLE_API_KEY ---
try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    GOOGLE_API_KEY = user_secrets.get_secret("GOOGLE_API_KEY")
except Exception as e:
    raise RuntimeError(
        "Could not read GOOGLE_API_KEY from Kaggle Secrets. "
        "Go to 'Add-ons' > 'Secrets' and create a secret named GOOGLE_API_KEY."
    ) from e

if not GOOGLE_API_KEY:
    raise RuntimeError("GOOGLE_API_KEY is empty. Please set it in Kaggle Secrets.")

# Configure Gemini client (Gemini Developer API with API key)
client = genai.Client(api_key=GOOGLE_API_KEY)

# --- Logging / Observability setup ---

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] [%(name)s] %(message)s",
)
logger = logging.getLogger("trip_planner")

# Simple in-memory metrics (observability)
METRICS = {
    "total_user_queries": 0,
    "total_agent_calls": 0,
    "total_tool_calls": 0,
    "places_api_calls": 0,
    "directions_api_calls": 0,
}

def log_metric(key: str, inc: int = 1):
    if key not in METRICS:
        METRICS[key] = 0
    METRICS[key] += inc

logger.info("Config and logging initialized successfully.")


2025-11-18 13:41:38,495 [INFO] [trip_planner] Config and logging initialized successfully.


In [9]:
# Cell 3: Google Places & Directions API tools

PLACES_TEXT_SEARCH_URL = "https://maps.googleapis.com/maps/api/place/textsearch/json"
DIRECTIONS_URL = "https://maps.googleapis.com/maps/api/directions/json"

def google_places_search(
    city: str,
    query: str,
    place_type: Optional[str] = None,
    max_results: int = 5,
) -> List[Dict[str, Any]]:
    """
    Tool: Search for places (hotels, attractions, restaurants, etc.) in a city.
    Uses Google Places Text Search API.
    """
    log_metric("total_tool_calls")
    log_metric("places_api_calls")
    
    params = {
        "key": GOOGLE_API_KEY,
        "query": f"{query} in {city}",
    }
    if place_type:
        params["type"] = place_type

    resp = requests.get(PLACES_TEXT_SEARCH_URL, params=params)
    resp.raise_for_status()
    data = resp.json()

    results = []
    for r in data.get("results", [])[:max_results]:
        results.append({
            "name": r.get("name"),
            "address": r.get("formatted_address"),
            "rating": r.get("rating"),
            "user_ratings_total": r.get("user_ratings_total"),
            "price_level": r.get("price_level"),
            "place_id": r.get("place_id"),
            "location": r.get("geometry", {}).get("location", {}),
            "types": r.get("types", []),
        })
    return results


def google_directions(
    origin: str,
    destination: str,
    mode: str = "driving",
) -> Dict[str, Any]:
    """
    Tool: Get directions & approximate travel time between two places.
    Uses Google Directions API.
    origin & destination can be text addresses or "lat,lng".
    """
    log_metric("total_tool_calls")
    log_metric("directions_api_calls")

    params = {
        "key": GOOGLE_API_KEY,
        "origin": origin,
        "destination": destination,
        "mode": mode,
    }

    resp = requests.get(DIRECTIONS_URL, params=params)
    resp.raise_for_status()
    data = resp.json()
    if not data.get("routes"):
        return {"duration_text": None, "distance_text": None}

    leg = data["routes"][0]["legs"][0]
    return {
        "duration_text": leg.get("duration", {}).get("text"),
        "distance_text": leg.get("distance", {}).get("text"),
        "summary": data["routes"][0].get("summary"),
    }


In [10]:
# Cell 4: Session & memory

@dataclass
class TripPreferences:
    budget_level: str = "medium"   # "low" / "medium" / "high"
    hotel_type: str = "hotel"      # "hostel", "hotel", "apartment"
    pace: str = "moderate"         # "relaxed", "moderate", "packed"
    interests: List[str] = field(default_factory=lambda: ["sightseeing", "local food"])
    notes: str = ""


@dataclass
class TripSessionState:
    user_id: str
    user_name: str = "Traveler"
    home_city: str = ""
    preferred_countries: List[str] = field(default_factory=list)
    past_destinations: List[str] = field(default_factory=list)
    preferences: TripPreferences = field(default_factory=TripPreferences)


# Very simple in-memory session store (for the notebook)
SESSIONS: Dict[str, TripSessionState] = {}

def get_session(user_id: str = "default") -> TripSessionState:
    if user_id not in SESSIONS:
        SESSIONS[user_id] = TripSessionState(user_id=user_id)
        logger.info(f"Created new session for user_id={user_id}")
    return SESSIONS[user_id]


def update_preferences_from_text(
    session: TripSessionState,
    trip_request: str,
    model_name: str = "gemini-2.0-flash",
):
    """
    Use Gemini to parse budget, pace, and interests from user's trip description.
    This is an example of "context engineering + memory".
    """
    log_metric("total_agent_calls")
    prompt = f"""
You are a parser. Extract the following fields from the user's trip request:

- budget_level: one of ["low", "medium", "high"]
- hotel_type: one of ["hostel", "hotel", "apartment"]
- pace: one of ["relaxed", "moderate", "packed"]
- interests: a list of 3-6 short tags like ["beach", "nightlife", "museums", "food"]

Return STRICTLY a JSON object with keys: budget_level, hotel_type, pace, interests.

User request: {trip_request}
"""
    response = client.models.generate_content(
        model=model_name,
        contents=prompt,
    )
    text = response.text
    try:
        # Try to extract JSON (Gemini often returns valid JSON if asked strongly)
        start = text.find("{")
        end = text.rfind("}")
        json_str = text[start:end+1]
        parsed = json.loads(json_str)

        prefs = session.preferences
        prefs.budget_level = parsed.get("budget_level", prefs.budget_level)
        prefs.hotel_type = parsed.get("hotel_type", prefs.hotel_type)
        prefs.pace = parsed.get("pace", prefs.pace)
        if isinstance(parsed.get("interests"), list):
            prefs.interests = parsed["interests"]

        logger.info(f"Updated preferences from text: {prefs}")
    except Exception as e:
        logger.warning(f"Could not parse preferences JSON from Gemini response: {e}\n{text}")


In [11]:
# Cell 5: Base agent using Gemini

class BaseAgent:
    def __init__(self, name: str, system_prompt: str, model_name: str = "gemini-2.0-flash"):
        self.name = name
        self.system_prompt = system_prompt
        self.model_name = model_name
        self.logger = logging.getLogger(name)

    def call_llm(self, prompt: str) -> str:
        """
        Call Gemini with a combined system + user prompt.
        """
        log_metric("total_agent_calls")
        full_prompt = f"{self.system_prompt.strip()}\n\nUser request:\n{prompt}"
        self.logger.info("Calling LLM...")
        response = client.models.generate_content(
            model=self.model_name,
            contents=full_prompt,
        )
        return response.text.strip()


In [12]:
# Cell 6: HotelsAgent

class HotelsAgent(BaseAgent):
    def __init__(self, model_name: str = "gemini-2.0-flash"):
        system_prompt = """
You are a Travel Accommodation Planning Agent.
You receive:
- city
- date range
- budget_level ("low", "medium", "high")
- hotel_type ("hostel", "hotel", "apartment")
- a list of candidate places from Google Places API with fields: name, address, rating, price_level, user_ratings_total.

Your job:
1. Pick 3–5 of the best options that match budget & hotel_type.
2. Explain briefly why each is a good choice.
3. Return a concise, user-friendly summary.
"""
        super().__init__("HotelsAgent", system_prompt, model_name)

    def plan_hotels(
        self,
        city: str,
        start_date: str,
        end_date: str,
        prefs: TripPreferences,
    ) -> str:
        # Use Google Places to fetch lodging options
        query = f"{prefs.hotel_type} in {city}"
        places = google_places_search(
            city=city,
            query=query,
            place_type="lodging",
            max_results=8,
        )

        context = json.dumps(places, indent=2)
        user_prompt = f"""
City: {city}
Dates: {start_date} to {end_date}
Desired budget level: {prefs.budget_level}
Preferred lodging type: {prefs.hotel_type}

Candidate places from Google Places:
{context}

Please pick the best 3-5 options and explain.
"""
        return self.call_llm(user_prompt), places


In [13]:
# Cell 7: ActivitiesAgent

class ActivitiesAgent(BaseAgent):
    def __init__(self, model_name: str = "gemini-2.0-flash"):
        system_prompt = """
You are a Daily Activities & Sightseeing Planning Agent.
You receive:
- city
- number of days
- trip preferences (budget_level, pace, interests)
- candidate places (attractions, restaurants, nightlife) from Google Places API

Your job:
1. Create a DAY-BY-DAY itinerary.
2. For each day:
   - Morning / Afternoon / Evening activities.
   - Use specific places from the candidate list when possible.
3. Respect user preferences (pace, interests, budget).
4. Keep it realistic: 3–5 activities per day.
"""
        super().__init__("ActivitiesAgent", system_prompt, model_name)

    def plan_activities(
        self,
        city: str,
        num_days: int,
        prefs: TripPreferences,
    ) -> str:
        # Gather attractions based on interest tags
        all_places = []

        interest_queries = []
        for interest in prefs.interests:
            if "beach" in interest.lower():
                interest_queries.append("beach")
            elif "nightlife" in interest.lower():
                interest_queries.append("bars and clubs")
            elif "museum" in interest.lower():
                interest_queries.append("museums")
            elif "food" in interest.lower():
                interest_queries.append("local restaurants")
            else:
                interest_queries.append(interest)

        # deduplicate
        interest_queries = list(dict.fromkeys(interest_queries))

        for q in interest_queries:
            try:
                places = google_places_search(
                    city=city,
                    query=q,
                    max_results=6,
                )
                all_places.extend(places)
            except Exception as e:
                self.logger.warning(f"Places search failed for query={q}: {e}")

        context = json.dumps(all_places[:40], indent=2)
        user_prompt = f"""
City: {city}
Number of days: {num_days}

Trip preferences:
- budget_level: {prefs.budget_level}
- pace: {prefs.pace}
- interests: {prefs.interests}

Candidate places (attractions, restaurants, nightlife):
{context}

Please create a realistic DAY-BY-DAY itinerary, using specific places when appropriate.
Format clearly with headings: Day 1, Day 2, etc.
"""
        return self.call_llm(user_prompt), all_places


In [14]:
# Cell 8: RoutingAgent (optional travel-time annotations)

class RoutingAgent(BaseAgent):
    def __init__(self, model_name: str = "gemini-2.0-flash"):
        system_prompt = """
You are a Routing & Logistics Agent.
You receive:
- city
- a day-wise itinerary with specific places
- (optionally) approximate travel times between consecutive spots

Your job:
1. Suggest minor adjustments to reduce back-and-forth travel.
2. Insert rough travel time annotations between stops.
3. Keep the final plan very readable.

If travel times are not provided, simply mention approximate travel times based on typical city trips.
"""
        super().__init__("RoutingAgent", system_prompt, model_name)

    def improve_itinerary_with_routing(
        self,
        city: str,
        itinerary_text: str,
    ) -> str:
        # For simplicity, we will not call Directions API for every pair (would be many calls).
        # Instead, we just pass the itinerary to Gemini and ask it to comment & slightly optimize.
        user_prompt = f"""
City: {city}

Here is an initial itinerary (per day, with activities):

{itinerary_text}

Please lightly optimize for route efficiency and add approximate travel-time notes.
"""
        return self.call_llm(user_prompt)


In [15]:
# Cell 9: Orchestrator (TripPlannerOrchestrator)

class TripPlannerOrchestrator(BaseAgent):
    def __init__(self, model_name: str = "gemini-2.0-flash"):
        system_prompt = """
You are the main Trip Planning Orchestrator.
You coordinate:
- HotelsAgent (for where to stay)
- ActivitiesAgent (for daily plan)
- RoutingAgent (for minor route optimization)

Given:
- city
- number of days
- date range
- trip preferences
- hotel summary
- activities itinerary (possibly improved by RoutingAgent)

Your job:
1. Combine everything into one clean trip plan.
2. Start with a short overview.
3. Then sections:
   - Trip Summary
   - Where to Stay
   - Day-by-Day Plan
   - Tips (budget, local culture, safety, transport)
4. Keep the tone friendly and practical.
"""
        super().__init__("TripPlannerOrchestrator", system_prompt, model_name)
        self.hotels_agent = HotelsAgent(model_name)
        self.activities_agent = ActivitiesAgent(model_name)
        self.routing_agent = RoutingAgent(model_name)

    def plan_trip(
        self,
        session: TripSessionState,
        city: str,
        num_days: int,
        start_date: str,
        end_date: str,
        user_free_text: str,
    ) -> str:
        logger.info(
            f"Orchestrator starting trip plan for {city}, {num_days} days, "
            f"{start_date} to {end_date}, user_id={session.user_id}"
        )

        # 1. Update preferences (memory) based on the user's description
        update_preferences_from_text(session, user_free_text)

        # 2. Hotels plan
        hotel_summary, hotels_raw = self.hotels_agent.plan_hotels(
            city=city,
            start_date=start_date,
            end_date=end_date,
            prefs=session.preferences,
        )

        # 3. Activities plan
        activities_summary, activities_raw = self.activities_agent.plan_activities(
            city=city,
            num_days=num_days,
            prefs=session.preferences,
        )

        # 4. Routing improvement (optional)
        improved_itinerary = self.routing_agent.improve_itinerary_with_routing(
            city=city,
            itinerary_text=activities_summary,
        )

        # 5. Combine everything via orchestrator LLM call
        combined_prompt = f"""
User: {session.user_name}
Home city: {session.home_city or '[unknown]'}
Past destinations: {session.past_destinations}
Preferences:
  - budget_level: {session.preferences.budget_level}
  - hotel_type: {session.preferences.hotel_type}
  - pace: {session.preferences.pace}
  - interests: {session.preferences.interests}

Trip request from user (free text):
{user_free_text}

Trip details:
- Destination city: {city}
- Number of days: {num_days}
- Dates: {start_date} to {end_date}

=== HotelsAgent summary ===
{hotel_summary}

=== ActivitiesAgent initial itinerary ===
{activities_summary}

=== RoutingAgent improved itinerary ===
{improved_itinerary}

Now, please synthesize all of this into ONE final trip plan with sections:
1. Trip Summary
2. Where to Stay
3. Day-by-Day Plan
4. Tips & Notes

Use headings and bullet points so it is easy to read.
"""
        final_plan = self.call_llm(combined_prompt)

        # 6. Update session long-term memory (simple example)
        if city not in session.past_destinations:
            session.past_destinations.append(city)

        logger.info("Trip plan generation complete.")
        return final_plan


In [16]:
# Cell 10: High-level helper to call the orchestrator

orchestrator = TripPlannerOrchestrator(model_name="gemini-2.0-flash")  # adjust model if needed

def plan_trip_for_user(
    user_id: str,
    city: str,
    num_days: int,
    start_date: str,
    end_date: str,
    description: str,
) -> str:
    """
    High-level function for interactive use.
    """
    log_metric("total_user_queries")
    session = get_session(user_id=user_id)
    result = orchestrator.plan_trip(
        session=session,
        city=city,
        num_days=num_days,
        start_date=start_date,
        end_date=end_date,
        user_free_text=description,
    )
    return result


In [17]:
# Cell 11: Manual test – plan a sample trip

sample_plan = plan_trip_for_user(
    user_id="devansh",
    city="Goa, India",
    num_days=3,
    start_date="2026-01-10",
    end_date="2026-01-13",
    description=(
        "I want a 3-day budget-friendly trip in Goa with beaches in the day "
        "and nightlife in the evening. I prefer hostels and a relaxed pace."
    ),
)

print(sample_plan)


2025-11-18 13:43:03,192 [INFO] [trip_planner] Created new session for user_id=devansh
2025-11-18 13:43:03,194 [INFO] [trip_planner] Orchestrator starting trip plan for Goa, India, 3 days, 2026-01-10 to 2026-01-13, user_id=devansh
2025-11-18 13:43:03,195 [INFO] [google_genai.models] AFC is enabled with max remote calls: 10.
2025-11-18 13:43:04,098 [INFO] [httpx] HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent "HTTP/1.1 200 OK"
2025-11-18 13:43:04,102 [INFO] [trip_planner] Updated preferences from text: TripPreferences(budget_level='low', hotel_type='hostel', pace='relaxed', interests=['beach', 'nightlife'], notes='')
2025-11-18 13:43:04,199 [INFO] [HotelsAgent] Calling LLM...
2025-11-18 13:43:04,200 [INFO] [google_genai.models] AFC is enabled with max remote calls: 10.
2025-11-18 13:43:07,237 [INFO] [httpx] HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent "HTTP/1.1 200 

Okay, here's your personalized 3-day Goa itinerary, designed for a budget-friendly, relaxed trip focusing on beaches and nightlife!

**Trip Summary**

Get ready for a fantastic Goan getaway! This 3-day itinerary balances relaxing on beautiful beaches with experiencing the vibrant nightlife that Goa is famous for. You'll explore North and South Goa, soak up the sun, enjoy local cuisine, and dance the night away, all while keeping your budget in mind. Remember that this itinerary is a suggestion, so feel free to adjust it to your liking.

**Where to Stay**

Since I don't have specific hostel recommendations, I recommend searching online for hostels in North Goa (Calangute/Candolim/Anjuna/Vagator) and South Goa (Palolem) that fit your budget and preferences. Key things to look for include:

*   **Location:** Consider areas close to beaches and nightlife spots.
*   **High ratings (4.0 or higher):** Indicating good overall guest satisfaction.
*   **A significant number of reviews (100+):** 

In [18]:
# Cell 12: Simple evaluation of the trip planner

from pprint import pprint

EVAL_CASES = [
    {
        "name": "Goa budget 3d nightlife",
        "user_id": "eval_user_goa",
        "city": "Goa, India",
        "num_days": 3,
        "start_date": "2026-01-10",
        "end_date": "2026-01-13",
        "description": "3 days in Goa, low budget, beaches and nightlife, hostels, relaxed pace.",
        "expected_keywords": ["Goa", "hostel", "beach", "nightlife"],
    },
    {
        "name": "Tokyo family 5d",
        "user_id": "eval_user_tokyo",
        "city": "Tokyo, Japan",
        "num_days": 5,
        "start_date": "2026-04-05",
        "end_date": "2026-04-10",
        "description": "5 days in Tokyo with kids, medium budget, museums, theme parks, and kid-friendly food.",
        "expected_keywords": ["Tokyo", "family", "kids", "museum"],
    },
]

def evaluate_plan(text: str, num_days: int, expected_keywords: List[str]) -> Dict[str, Any]:
    """
    Very simple heuristic evaluator:
    - Checks presence of 'Day 1', 'Day 2', etc.
    - Counts how many expected keywords appear.
    """
    scores = {}

    # Day headings
    day_hits = 0
    for d in range(1, num_days + 1):
        if f"Day {d}" in text or f"Day {d}:" in text:
            day_hits += 1
    scores["day_coverage"] = day_hits / num_days

    # Keywords
    kw_hits = 0
    lower_text = text.lower()
    for kw in expected_keywords:
        if kw.lower() in lower_text:
            kw_hits += 1
    scores["keyword_coverage"] = kw_hits / max(1, len(expected_keywords))

    # Simple aggregate
    scores["overall_score"] = (scores["day_coverage"] + scores["keyword_coverage"]) / 2
    return scores


all_eval_results = []

for case in EVAL_CASES:
    print(f"\n=== Running eval case: {case['name']} ===")
    plan = plan_trip_for_user(
        user_id=case["user_id"],
        city=case["city"],
        num_days=case["num_days"],
        start_date=case["start_date"],
        end_date=case["end_date"],
        description=case["description"],
    )
    scores = evaluate_plan(
        text=plan,
        num_days=case["num_days"],
        expected_keywords=case["expected_keywords"],
    )
    result = {
        "case_name": case["name"],
        "scores": scores,
    }
    all_eval_results.append(result)
    pprint(result)

print("\n=== Aggregated Evaluation Results ===")
pprint(all_eval_results)


2025-11-18 13:44:07,203 [INFO] [trip_planner] Created new session for user_id=eval_user_goa
2025-11-18 13:44:07,204 [INFO] [trip_planner] Orchestrator starting trip plan for Goa, India, 3 days, 2026-01-10 to 2026-01-13, user_id=eval_user_goa
2025-11-18 13:44:07,205 [INFO] [google_genai.models] AFC is enabled with max remote calls: 10.



=== Running eval case: Goa budget 3d nightlife ===


2025-11-18 13:44:08,167 [INFO] [httpx] HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent "HTTP/1.1 200 OK"
2025-11-18 13:44:08,170 [INFO] [trip_planner] Updated preferences from text: TripPreferences(budget_level='low', hotel_type='hostel', pace='relaxed', interests=['beach', 'nightlife'], notes='')
2025-11-18 13:44:08,264 [INFO] [HotelsAgent] Calling LLM...
2025-11-18 13:44:08,265 [INFO] [google_genai.models] AFC is enabled with max remote calls: 10.
2025-11-18 13:44:12,225 [INFO] [httpx] HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent "HTTP/1.1 200 OK"
2025-11-18 13:44:12,412 [INFO] [ActivitiesAgent] Calling LLM...
2025-11-18 13:44:12,413 [INFO] [google_genai.models] AFC is enabled with max remote calls: 10.
2025-11-18 13:44:21,375 [INFO] [httpx] HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent "HTTP/1.1 200 O

{'case_name': 'Goa budget 3d nightlife',
 'scores': {'day_coverage': 1.0, 'keyword_coverage': 1.0, 'overall_score': 1.0}}

=== Running eval case: Tokyo family 5d ===


2025-11-18 13:44:37,363 [INFO] [httpx] HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent "HTTP/1.1 200 OK"
2025-11-18 13:44:37,366 [INFO] [trip_planner] Updated preferences from text: TripPreferences(budget_level='medium', hotel_type='hotel', pace='moderate', interests=['museums', 'theme parks', 'food'], notes='')
2025-11-18 13:44:37,445 [INFO] [HotelsAgent] Calling LLM...
2025-11-18 13:44:37,446 [INFO] [google_genai.models] AFC is enabled with max remote calls: 10.
2025-11-18 13:44:44,531 [INFO] [httpx] HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent "HTTP/1.1 200 OK"
2025-11-18 13:44:44,780 [INFO] [ActivitiesAgent] Calling LLM...
2025-11-18 13:44:44,781 [INFO] [google_genai.models] AFC is enabled with max remote calls: 10.
2025-11-18 13:44:49,979 [INFO] [httpx] HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent 

{'case_name': 'Tokyo family 5d',
 'scores': {'day_coverage': 1.0, 'keyword_coverage': 1.0, 'overall_score': 1.0}}

=== Aggregated Evaluation Results ===
[{'case_name': 'Goa budget 3d nightlife',
  'scores': {'day_coverage': 1.0,
             'keyword_coverage': 1.0,
             'overall_score': 1.0}},
 {'case_name': 'Tokyo family 5d',
  'scores': {'day_coverage': 1.0,
             'keyword_coverage': 1.0,
             'overall_score': 1.0}}]


In [19]:
# Cell 13: Print metrics (observability demo)

print("=== Trip Planner Metrics ===")
for k, v in METRICS.items():
    print(f"{k}: {v}")


=== Trip Planner Metrics ===
total_user_queries: 3
total_agent_calls: 15
total_tool_calls: 10
places_api_calls: 10
directions_api_calls: 0


In [25]:
# NEW Cell 14: Simpler chat loop (no LLM parsing – fewer API calls)

def chat_loop_simple(user_id: str = "devansh"):
    """
    Simple REPL-style chat loop WITHOUT extra LLM parsing.
    You: describe your trip
    Agent: asks for city, days, dates, then returns full plan.

    Type 'exit' or 'quit' to end.
    """
    print("=== Trip Planner Chat (Simple) ===")
    print("Type 'exit' or 'quit' at any time to stop.\n")

    while True:
        description = input("Describe your trip (budget, interests, hotel type, pace):\nYou: ").strip()
        if description.lower() in ["exit", "quit"]:
            print("TripPlanner: Bye! 👋")
            break
        if not description:
            continue

        city = input("Destination city (e.g. 'Goa, India'): ").strip()
        if city.lower() in ["exit", "quit"]:
            print("TripPlanner: Bye! 👋")
            break

        num_days_str = input("Number of days (e.g. 3): ").strip()
        if num_days_str.lower() in ["exit", "quit"]:
            print("TripPlanner: Bye! 👋")
            break
        try:
            num_days = int(num_days_str)
        except ValueError:
            print("Please enter a valid integer for number of days. Try again.\n")
            continue

        start_date = input("Start date (YYYY-MM-DD): ").strip()
        if start_date.lower() in ["exit", "quit"]:
            print("TripPlanner: Bye! 👋")
            break

        end_date = input("End date (YYYY-MM-DD): ").strip()
        if end_date.lower() in ["exit", "quit"]:
            print("TripPlanner: Bye! 👋")
            break

        print("\nTripPlanner: Planning your trip... (this will use the Gemini API)\n")

        plan = plan_trip_for_user(
            user_id=user_id,
            city=city,
            num_days=num_days,
            start_date=start_date,
            end_date=end_date,
            description=description,
        )

        print("========== TRIP PLAN ==========\n")
        print(plan)
        print("\n===============================\n")


# Run the simpler chat loop
chat_loop_simple(user_id="devansh")


=== Trip Planner Chat (Simple) ===
Type 'exit' or 'quit' at any time to stop.



Describe your trip (budget, interests, hotel type, pace):
You:  200000,party,villa,smooth
Destination city (e.g. 'Goa, India'):  barcelona
Number of days (e.g. 3):  4
Start date (YYYY-MM-DD):  2026-02-23
End date (YYYY-MM-DD):  2026-02-27


2025-11-18 13:55:54,646 [INFO] [trip_planner] Orchestrator starting trip plan for barcelona, 4 days, 2026-02-23 to 2026-02-27, user_id=devansh
2025-11-18 13:55:54,647 [INFO] [google_genai.models] AFC is enabled with max remote calls: 10.



TripPlanner: Planning your trip... (this will use the Gemini API)



2025-11-18 13:55:55,594 [INFO] [httpx] HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent "HTTP/1.1 200 OK"
2025-11-18 13:55:55,597 [INFO] [trip_planner] Updated preferences from text: TripPreferences(budget_level='high', hotel_type='apartment', pace='relaxed', interests=['party'], notes='')
2025-11-18 13:55:55,682 [INFO] [HotelsAgent] Calling LLM...
2025-11-18 13:55:55,682 [INFO] [google_genai.models] AFC is enabled with max remote calls: 10.
2025-11-18 13:55:59,014 [INFO] [httpx] HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent "HTTP/1.1 200 OK"
2025-11-18 13:55:59,098 [INFO] [ActivitiesAgent] Calling LLM...
2025-11-18 13:55:59,098 [INFO] [google_genai.models] AFC is enabled with max remote calls: 10.
2025-11-18 13:56:05,539 [INFO] [httpx] HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent "HTTP/1.1 200 OK"
2025-1

========== TRIP PLAN ==========

Okay, here's your comprehensive Barcelona party trip plan for February 23-27, 2026. Get ready for a high-end experience filled with luxury, culture, and unforgettable nightlife!

## Barcelona: Luxury Party Trip (Feb 23-27, 2026)

**Overview:**

This 4-day itinerary focuses on providing a relaxed yet vibrant experience in Barcelona, tailored for a traveler with a high budget and an interest in partying. You'll explore upscale shopping, exclusive nightlife, cultural landmarks, and gourmet dining, all while enjoying the comfort and luxury Barcelona has to offer.

## Trip Summary

*   **Destination:** Barcelona, Spain
*   **Dates:** February 23 - February 27, 2026 (4 days)
*   **Budget:** High
*   **Interests:** Party, Luxury, Culture, Relaxed Pace
*   **Accommodation:** Apartment (specific recommendations pending Google Places data)

## Where to Stay

*   **Accommodation Type:** Luxury Apartment/Villa
*   **Neighborhood Recommendations:** Gothic Quarter, E

Describe your trip (budget, interests, hotel type, pace):
You:  quit


TripPlanner: Bye! 👋
